In [12]:
!pip install -U sentence-transformers -Uq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import pickle

In [14]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[ 6.76536411e-02  6.34698719e-02  4.87222448e-02  7.93291703e-02
   3.74474451e-02  2.65416852e-03  3.93871330e-02 -7.05680298e-03
   5.93382567e-02  3.15077677e-02  6.01203702e-02 -5.29313534e-02
   4.05598916e-02 -2.59261802e-02  2.98265498e-02  1.12700148e-03
   7.35719353e-02 -5.03596738e-02 -1.22384027e-01  2.36648731e-02
   2.96844803e-02  4.25143242e-02  2.56668404e-02  1.97289372e-03
  -5.69618382e-02 -2.71008704e-02 -3.29472534e-02  6.60447106e-02
   1.18976191e-01 -4.58509140e-02 -7.25975260e-02 -3.26423943e-02
   5.23202941e-02  4.50059399e-02  8.26475024e-03  3.67564261e-02
  -1.38691971e-02  6.53574541e-02 -2.64232829e-02  1.00321864e-04
  -1.36748571e-02 -3.63879651e-02 -1.94556098e-02 -2.90018581e-02
   3.94659825e-02 -8.84063020e-02  2.61367555e-03  1.36764450e-02
   4.83106561e-02 -3.11642326e-02 -1.17270075e-01 -5.11791185e-02
  -8.85436833e-02 -2.19217055e-02  1.43085849e-02  4.43920419e-02
  -1.34417415e-02  7.43395835e-02  2.66726650e-02 -1.98368710e-02
   1.79796

In [15]:
devops_concepts = [
    {
        "name": "Continuous Integration (CI)",
        "description": "CI is the practice of frequently integrating code changes into a shared repository. Each integration is automatically verified by building and running automated tests, enabling early detection of problems.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Continuous Delivery (CD)",
        "description": "CD extends CI by automatically deploying all code changes to a testing or production environment after the build stage. This ensures that every change is ready for release at any time.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Infrastructure as Code (IaC)",
        "description": "IaC involves managing and provisioning computing infrastructure through machine-readable definition files, rather than physical hardware configuration or interactive configuration tools.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Microservices",
        "description": "Microservices is an architectural style that structures an application as a collection of small, loosely coupled services. Each service is independently deployable and scalable.",
        "priority": "Medium",
        "difficulty": "Medium"
    },
    {
        "name": "Containerization",
        "description": "Containerization is a lightweight form of virtualization where applications are packaged with their dependencies and run in isolated environments called containers.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Orchestration",
        "description": "Orchestration involves automating the deployment, scaling, and management of containerized applications. Popular orchestrators include Kubernetes, Docker Swarm, and Apache Mesos.",
        "priority": "High",
        "difficulty": "High"
    },
    {
        "name": "Monitoring and Logging",
        "description": "Monitoring involves observing the performance and availability of systems and applications, while logging captures and stores events and data for analysis and troubleshooting.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Configuration Management",
        "description": "Configuration management is the process of maintaining consistent and reliable performance by managing the configurations of infrastructure and applications.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Deployment Strategies",
        "description": "Deployment strategies define how new versions of an application are released to users. Examples include blue-green deployment, canary deployment, and rolling deployment.",
        "priority": "High",
        "difficulty": "Medium"
    },
    {
        "name": "Collaboration and Communication",
        "description": "Effective collaboration and communication are essential in DevOps culture to foster cooperation between development, operations, and other stakeholders.",
        "priority": "High",
        "difficulty": "Low"
    }
]

In [16]:
# devops_em = model.encode(devops_concepts)
# with open('devops_em.pkl', 'wb') as file:
#     pickle.dump(devops_em, file)


In [17]:
with open('devops_em.pkl', 'rb') as file:
    devops_em = pickle.load(file)

In [18]:
def rag(query):
    q_embed = model.encode(query)
    sim = util.cos_sim(q_embed, devops_em)
    sorted_sim = sim.sort(descending=True)
    print(devops_concepts[sorted_sim[1][0][0]])

In [19]:
rag("how to setup something?")

{'name': 'Configuration Management', 'description': 'Configuration management is the process of maintaining consistent and reliable performance by managing the configurations of infrastructure and applications.', 'priority': 'High', 'difficulty': 'Medium'}


In [20]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
def mistral(query):
    
    context = rag(query)
    context = f'''{context}'''
#     print(context)
    prompt = f"You an intelligent agent who can answer a question from the context, question:{query} context:{context}"    
    openai_api_key = "EMPTY"
    openai_api_base = "http://dgx3.humanbrain.in:50000/v1/"

    client = OpenAI(
        api_key=openai_api_key,
        base_url=openai_api_base,
    )
    print("something")

    chat_response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream = True,
        temperature = 0,
        max_tokens = 3000
    )

    for chunk in chat_response:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="")

In [21]:
mistral("how rob?")

{'name': 'Continuous Integration (CI)', 'description': 'CI is the practice of frequently integrating code changes into a shared repository. Each integration is automatically verified by building and running automated tests, enabling early detection of problems.', 'priority': 'High', 'difficulty': 'Medium'}
something
 I'm an intelligent agent designed to answer questions. However, I need a valid question to answer. The context provided does not contain any information that can be used to form a question. Could you please provide a specific question so I can help you with that?

In [22]:
mistral("how to create a container in devops?")

{'name': 'Containerization', 'description': 'Containerization is a lightweight form of virtualization where applications are packaged with their dependencies and run in isolated environments called containers.', 'priority': 'High', 'difficulty': 'Medium'}
something
 To create a container in DevOps, you can use various tools and platforms, such as Docker, Kubernetes, or Amazon Elastic Container Service (ECS). Here's a general step-by-step guide using Docker:

1. Install Docker: Before creating a container, you need to have Docker installed on your system. You can download Docker from the official website (https://www.docker.com/get-started) and follow the installation instructions for your operating system.

2. Write a Dockerfile: A Dockerfile is a text document that contains all the instructions needed to build a Docker image. You can write a Dockerfile using a text editor or IDE of your choice. Here's an example Dockerfile for a simple Node.js application:

```Dockerfile
# Use an offi